In [2]:
import numpy as np
from math import cos, sin

coordination = np.array([[-86.15, -68.99, 36589.41, 25273.32, 2195.17],
                         [-53.40, 82.21, 37631.08, 31324.51, 728.69],
                         [-14.78, -76.63, 39100.97, 24934.98, 2386.50],
                         [10.46, 64.43, 40426.54, 30319.81, 757.31]])

m = 50
f = 153.24


phi = 0
omega = 0
kappa = 0
Z_S0 = m*f
X_S0 = np.mean(coordination[:, 2])
Y_S0 = np.mean(coordination[:, 3])

X = np.array([X_S0, Y_S0, Z_S0, phi, omega, kappa])



print(X)


[38437.    27963.155  7662.        0.        0.        0.   ]


In [3]:
R = np.array([[cos(X[3])*cos(X[5]) - sin(X[3])*sin(X[4])*sin(X[5]), -cos(X[3])*sin(X[5]) - sin(X[3])*sin(X[4])*sin(X[5]), -sin(X[3])*cos(X[3])], 
              [cos(X[4])*sin(X[5]), cos(X[4])*cos(X[5]), -sin(X[4])], 
              [sin(X[3])*cos(X[5]) + cos(X[3])*sin(X[4])*cos(X[5]), -sin(X[3])*sin(X[5]) + cos(X[3])*sin(X[4])*cos(X[5]), cos(X[3])*cos(X[4])]])



print(R)

[[ 1. -0. -0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]


In [4]:
bar = np.zeros((4,3))
for i in range(4):
    bar[i, 0] = R[0,:]@(coordination[i, 2:5]-X[0:3]).T
    bar[i, 1] = R[1,:]@(coordination[i, 2:5]-X[0:3]).T
    bar[i, 2] = R[2,:]@(coordination[i, 2:5]-X[0:3]).T

print(bar)

[[-1847.59  -2689.835 -5466.83 ]
 [ -805.92   3361.355 -6933.31 ]
 [  663.97  -3028.175 -5275.5  ]
 [ 1989.54   2356.655 -6904.69 ]]


In [5]:


L = np.zeros((1, 8))
for i in range(4):
    L[0, i*2] = coordination[i, 0] + f * bar[i,0] / bar[i,2]
    L[0, i*2+1] = coordination[i, 1] + f * bar[i,1] / bar[i,2]



print(L)

[[-34.36044525   6.40841469 -35.58755821   7.91734033 -34.06665772
   11.3308638  -33.69507569  12.12731701]]


In [6]:
A = np.zeros((8,6), dtype=float)
for i in range(4):
    A[i*2, 0:3] = (f*R[:,0] + R[:,2]*coordination[i, 0]) / bar[i,2]
    A[i*2+1, 0:3] = (f*R[:,1] + R[:,2]*coordination[i, 1]) / bar[i,2]

    A[i*2, 3] = coordination[i, 1]*sin(X[4]) - cos(X[4])*(f*cos(X[5])+(coordination[i, 0]*cos(X[5])-coordination[i, 1]*sin(X[5]))*coordination[i, 0]/f)
    A[i*2, 4] = - f*sin(X[5]) - (coordination[i, 0]*sin(X[5])+coordination[i, 1]*cos(X[5])*coordination[i, 0]/f)
    A[i*2, 5] = coordination[i, 1]
    A[i*2+1, 3] = - coordination[i, 0]*sin(X[4]) - cos(X[4]) * (coordination[i,1]*(coordination[i,0]*cos(X[5])-coordination[i,1]*sin(X[5]))/f - f*sin(X[5]))
    A[i*2+1, 4] = - f*cos(X[5]) - (L[0, 2*i]*sin(X[5])+coordination[i, 1]*cos(X[5])*coordination[i, 1]/f)
    A[i*2+1, 5] = - coordination[i, 0]



print(A)

[[-2.80308698e-02 -0.00000000e+00  1.57586755e-02 -2.01672671e+02
  -3.87854901e+01 -6.89900000e+01]
 [-0.00000000e+00 -2.80308698e-02  1.26197449e-02 -3.87854901e+01
  -1.84299907e+02  8.61500000e+01]
 [-2.21019975e-02 -0.00000000e+00  7.70194900e-03 -1.71848457e+02
   2.86479640e+01  8.22100000e+01]
 [ 0.00000000e+00 -2.21019975e-02 -1.18572514e-02  2.86479640e+01
  -1.97343916e+02  5.34000000e+01]
 [-2.90474837e-02 -0.00000000e+00  2.80163018e-03 -1.54665531e+02
  -7.39096450e+00 -7.66300000e+01]
 [-0.00000000e+00 -2.90474837e-02  1.45256374e-02 -7.39096450e+00
  -1.91560001e+02  1.47800000e+01]
 [-2.21936104e-02 -0.00000000e+00 -1.51491233e-03 -1.53953989e+02
  -4.39792352e+00  6.44300000e+01]
 [ 0.00000000e+00 -2.21936104e-02 -9.33133855e-03 -4.39792352e+00
  -1.80329695e+02 -1.04600000e+01]]


In [7]:
dX = np.linalg.inv(A.T@A)@A.T@L.T

print(dX)

[[ 1.36051787e+03]
 [-4.83171575e+02]
 [-1.01704686e+02]
 [-4.14461137e-03]
 [ 1.44481515e-03]
 [-6.66746927e-02]]
